In [1]:
import os
import random
import zlib
from itertools import combinations
import urllib.request
import zipfile

# ---------------------------------------------------------
# Helper Functions for K-Grams and Jaccard
# ---------------------------------------------------------
def get_kgrams_char(text, k):
    """Constructs k-grams based on characters (spaces count)."""
    return set([text[i:i+k] for i in range(len(text) - k + 1)])

def get_kgrams_word(text, k):
    """Constructs k-grams based on words."""
    words = text.split()
    return set([" ".join(words[i:i+k]) for i in range(len(words) - k + 1)])

def exact_jaccard(set1, set2):
    """Calculates exact Jaccard similarity."""
    if not set1 and not set2: return 1.0
    if not set1 or not set2: return 0.0
    return len(set1.intersection(set2)) / len(set1.union(set2))

# ---------------------------------------------------------
# MinHash Class Implementation
# ---------------------------------------------------------
class MinHash:
    def __init__(self, num_hashes, prime=4294967311, max_val=(2**32)-1):
        self.num_hashes = num_hashes
        self.prime = prime
        self.a = [random.randint(1, max_val) for _ in range(num_hashes)]
        self.b = [random.randint(0, max_val) for _ in range(num_hashes)]
        
    def _hash_string(self, string_val):
        return zlib.crc32(string_val.encode('utf-8')) & 0xffffffff

    def compute_signature(self, item_set):
        signature = [float('inf')] * self.num_hashes
        for item in item_set:
            hash_val = self._hash_string(item)
            for i in range(self.num_hashes):
                mapped_val = (self.a[i] * hash_val + self.b[i]) % self.prime
                if mapped_val < signature[i]:
                    signature[i] = mapped_val
        return signature

def approx_jaccard(sig1, sig2):
    """Calculates approximate Jaccard from MinHash signatures."""
    matches = sum(1 for i, j in zip(sig1, sig2) if i == j)
    return matches / len(sig1)

In [2]:
# Read the actual documents from the minhash folder
docs = {}
for i in range(1, 5):
    with open(f'minhash/D{i}.txt', 'r', encoding='utf-8') as f:
        # Read file, remove any accidental newlines, keep lowercase and spaces
        docs[f'D{i}'] = f.read().replace('\n', '')

print("--- QUESTION 1: k-Grams ---")
kgram_types = {
    '2-char': lambda t: get_kgrams_char(t, 2),
    '3-char': lambda t: get_kgrams_char(t, 3),
    '2-word': lambda t: get_kgrams_word(t, 2)
}

doc_kgrams = {k_type: {doc_id: func(text) for doc_id, text in docs.items()} 
              for k_type, func in kgram_types.items()}

pairs = list(combinations(['D1', 'D2', 'D3', 'D4'], 2))

for k_type in kgram_types.keys():
    print(f"\nExact Jaccard for {k_type} grams:")
    for d1, d2 in pairs:
        sim = exact_jaccard(doc_kgrams[k_type][d1], doc_kgrams[k_type][d2])
        print(f"  {d1} & {d2}: {sim:.4f}")

print("\n--- QUESTION 2: Min-Hashing D1 & D2 (using 3-char grams) ---")
d1_3grams = doc_kgrams['3-char']['D1']
d2_3grams = doc_kgrams['3-char']['D2']

t_values = [20, 60, 150, 300, 600]
for t in t_values:
    random.seed(42) # Ensure repeatable hashes
    mh = MinHash(num_hashes=t)
    sig1 = mh.compute_signature(d1_3grams)
    sig2 = mh.compute_signature(d2_3grams)
    approx_sim = approx_jaccard(sig1, sig2)
    print(f"Approx Jaccard for t={t:3d}: {approx_sim:.4f}")

--- QUESTION 1: k-Grams ---

Exact Jaccard for 2-char grams:
  D1 & D2: 0.9811
  D1 & D3: 0.8157
  D1 & D4: 0.6444
  D2 & D3: 0.8000
  D2 & D4: 0.6413
  D3 & D4: 0.6530

Exact Jaccard for 3-char grams:
  D1 & D2: 0.9780
  D1 & D3: 0.5804
  D1 & D4: 0.3051
  D2 & D3: 0.5680
  D2 & D4: 0.3059
  D3 & D4: 0.3121

Exact Jaccard for 2-word grams:
  D1 & D2: 0.9408
  D1 & D3: 0.1823
  D1 & D4: 0.0302
  D2 & D3: 0.1737
  D2 & D4: 0.0303
  D3 & D4: 0.0161

--- QUESTION 2: Min-Hashing D1 & D2 (using 3-char grams) ---
Approx Jaccard for t= 20: 0.9500
Approx Jaccard for t= 60: 0.9833
Approx Jaccard for t=150: 0.9800
Approx Jaccard for t=300: 0.9800
Approx Jaccard for t=600: 0.9800


In [3]:
print("--- QUESTION 3: LSH Probabilities ---")
# t = 160, tau = 0.7
# We found r=8, b=20 provides a threshold of (1/20)^(1/8) = 0.684 (Close to 0.7)
r = 8
b = 20

def s_curve_prob(s, r, b):
    return 1 - (1 - s**b)**r

print(f"Chosen values: b = {b}, r = {r}")
print("Probability of being estimated as candidates (3-char grams):")

for d1, d2 in pairs:
    sim = exact_jaccard(doc_kgrams['3-char'][d1], doc_kgrams['3-char'][d2])
    prob = s_curve_prob(sim, r, b)
    print(f"  {d1} & {d2} (Exact Sim: {sim:.4f}) -> Probability: {prob:.6f}")

--- QUESTION 3: LSH Probabilities ---
Chosen values: b = 20, r = 8
Probability of being estimated as candidates (3-char grams):
  D1 & D2 (Exact Sim: 0.9780) -> Probability: 0.999722
  D1 & D3 (Exact Sim: 0.5804) -> Probability: 0.000150
  D1 & D4 (Exact Sim: 0.3051) -> Probability: 0.000000
  D2 & D3 (Exact Sim: 0.5680) -> Probability: 0.000098
  D2 & D4 (Exact Sim: 0.3059) -> Probability: 0.000000
  D3 & D4 (Exact Sim: 0.3121) -> Probability: 0.000000


In [6]:
print("--- QUESTION 4: MovieLens Dataset Min-Hashing (OPTIMIZED) ---")

import os
import random
import zlib
import urllib.request
import zipfile
import time
from itertools import combinations

# 1. Download and Extract MovieLens
url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
if not os.path.exists("ml-100k"):
    print("Downloading MovieLens dataset...")
    urllib.request.urlretrieve(url, "ml-100k.zip")
    with zipfile.ZipFile("ml-100k.zip", 'r') as zip_ref:
        zip_ref.extractall(".")

# 2. Parse Data
user_movies = {}
all_movies = set()
with open("ml-100k/u.data", "r") as f:
    for line in f:
        user_id, movie_id, rating, timestamp = line.strip().split('\t')
        if user_id not in user_movies:
            user_movies[user_id] = set()
        user_movies[user_id].add(movie_id)
        all_movies.add(movie_id)

user_ids = list(user_movies.keys())

# --- OPTIMIZATIONS ---
# Fast Jaccard using basic set math
def exact_jaccard_fast(set1, set2):
    if not set1 or not set2: return 0.0
    intersection_len = len(set1 & set2)
    return intersection_len / (len(set1) + len(set2) - intersection_len)

# Pre-hash all movie strings ONCE instead of millions of times
print("Pre-hashing movie strings for MinHash speedup...")
movie_crc32 = {m: zlib.crc32(m.encode('utf-8')) & 0xffffffff for m in all_movies}
user_movie_hashes = {u: [movie_crc32[m] for m in user_movies[u]] for u in user_ids}

class FastMinHash:
    def __init__(self, num_hashes, prime=4294967311, max_val=(2**32)-1):
        self.num_hashes = num_hashes
        self.prime = prime
        self.a = [random.randint(1, max_val) for _ in range(num_hashes)]
        self.b = [random.randint(0, max_val) for _ in range(num_hashes)]
        
    def compute_signature(self, item_hashes):
        signature = [float('inf')] * self.num_hashes
        for hash_val in item_hashes:
            for i in range(self.num_hashes):
                mapped_val = (self.a[i] * hash_val + self.b[i]) % self.prime
                if mapped_val < signature[i]:
                    signature[i] = mapped_val
        return signature
# ---------------------

# 3. Calculate Exact Jaccard >= 0.5
exact_pairs_05 = set()
print("Calculating exact Jaccard for all 444,153 pairs...")
start_time = time.time()
for i in range(len(user_ids)):
    u1 = user_ids[i]
    for j in range(i + 1, len(user_ids)):
        u2 = user_ids[j]
        if exact_jaccard_fast(user_movies[u1], user_movies[u2]) >= 0.5:
            exact_pairs_05.add(tuple(sorted((u1, u2))))
print(f"Found {len(exact_pairs_05)} exact pairs in {time.time() - start_time:.2f} seconds.")

# 4. Run Min-Hashing 5 times
t_configs = [50, 100, 200]
for t in t_configs:
    print(f"\n--- Running 5 trials for t={t} hash functions ---")
    total_fp = 0
    total_fn = 0
    req_matches = 0.5 * t  # Math equivalent to similarity >= 0.5
    
    for run in range(5):
        run_start = time.time()
        mh = FastMinHash(num_hashes=t)
        user_signatures = {u: mh.compute_signature(user_movie_hashes[u]) for u in user_ids}
        
        approx_pairs = set()
        
        # Fast inner loop comparison
        for i in range(len(user_ids)):
            u1 = user_ids[i]
            sig1 = user_signatures[u1]
            for j in range(i + 1, len(user_ids)):
                u2 = user_ids[j]
                sig2 = user_signatures[u2]
                
                # Manual loop is faster than zip() for large datasets
                matches = 0
                for k in range(t):
                    if sig1[k] == sig2[k]:
                        matches += 1
                        
                if matches >= req_matches:
                    approx_pairs.add(tuple(sorted((u1, u2))))
                    
        fp = len(approx_pairs - exact_pairs_05)
        fn = len(exact_pairs_05 - approx_pairs)
        total_fp += fp
        total_fn += fn
        print(f"  Run {run+1}/5 finished in {time.time() - run_start:.2f}s | FP: {fp}, FN: {fn}")
        
    print(f"AVERAGE Results (t={t}): False Positives: {total_fp / 5} | False Negatives: {total_fn / 5}")

--- QUESTION 4: MovieLens Dataset Min-Hashing (OPTIMIZED) ---
Pre-hashing movie strings for MinHash speedup...
Calculating exact Jaccard for all 444,153 pairs...
Found 10 exact pairs in 2.20 seconds.

--- Running 5 trials for t=50 hash functions ---
  Run 1/5 finished in 4.45s | FP: 273, FN: 1
  Run 2/5 finished in 4.52s | FP: 165, FN: 2
  Run 3/5 finished in 4.17s | FP: 175, FN: 0
  Run 4/5 finished in 4.27s | FP: 81, FN: 5
  Run 5/5 finished in 4.07s | FP: 44, FN: 1
AVERAGE Results (t=50): False Positives: 147.6 | False Negatives: 1.8

--- Running 5 trials for t=100 hash functions ---
  Run 1/5 finished in 9.10s | FP: 17, FN: 1
  Run 2/5 finished in 8.94s | FP: 38, FN: 4
  Run 3/5 finished in 8.19s | FP: 44, FN: 2
  Run 4/5 finished in 8.20s | FP: 35, FN: 0
  Run 5/5 finished in 8.29s | FP: 14, FN: 2
AVERAGE Results (t=100): False Positives: 29.6 | False Negatives: 1.8

--- Running 5 trials for t=200 hash functions ---
  Run 1/5 finished in 16.21s | FP: 9, FN: 2
  Run 2/5 finished in

In [7]:
print("--- QUESTION 5: LSH on MovieLens (OPTIMIZED) ---")

def run_lsh_experiment_fast(target_sim, exact_pairs):
    lsh_configs = [
        (50, 5, 10),   # t=50, r=5, b=10
        (100, 5, 20),  # t=100, r=5, b=20
        (200, 5, 40),  # t=200, r=5, b=40
        (200, 10, 20)  # t=200, r=10, b=20
    ]

    for t, r, b in lsh_configs:
        total_fp = 0
        total_fn = 0
        req_matches = target_sim * t
        
        for run in range(5):
            mh = FastMinHash(num_hashes=t)
            user_signatures = {u: mh.compute_signature(user_movie_hashes[u]) for u in user_ids}
            
            # LSH bucket logic
            candidate_pairs = set()
            for band_idx in range(b):
                buckets = {}
                start = band_idx * r
                end = start + r
                for u, sig in user_signatures.items():
                    band_tuple = tuple(sig[start:end])
                    if band_tuple not in buckets: buckets[band_tuple] = []
                    buckets[band_tuple].append(u)
                
                for bucket_users in buckets.values():
                    if len(bucket_users) > 1:
                        # Only combinations within the same bucket
                        for i in range(len(bucket_users)):
                            for j in range(i + 1, len(bucket_users)):
                                candidate_pairs.add(tuple(sorted((bucket_users[i], bucket_users[j]))))
            
            # Filter candidates by approximate Jaccard >= target_sim
            final_pairs = set()
            for u1, u2 in candidate_pairs:
                sig1 = user_signatures[u1]
                sig2 = user_signatures[u2]
                
                matches = 0
                for k in range(t):
                    if sig1[k] == sig2[k]:
                        matches += 1
                        
                if matches >= req_matches:
                    final_pairs.add((u1, u2))
                    
            fp = len(final_pairs - exact_pairs)
            fn = len(exact_pairs - final_pairs)
            total_fp += fp
            total_fn += fn
            
        print(f"Config t={t}, r={r}, b={b}: Avg FP = {total_fp/5}, Avg FN = {total_fn/5}")

# 1. Calculate Exact targets for 0.6 and 0.8
print("Calculating exact Jaccard for 0.6 and 0.8 thresholds...")
exact_pairs_06 = set()
exact_pairs_08 = set()
for i in range(len(user_ids)):
    u1 = user_ids[i]
    for j in range(i + 1, len(user_ids)):
        u2 = user_ids[j]
        sim = exact_jaccard_fast(user_movies[u1], user_movies[u2])
        if sim >= 0.6: exact_pairs_06.add(tuple(sorted((u1, u2))))
        if sim >= 0.8: exact_pairs_08.add(tuple(sorted((u1, u2))))

print("\n--- Target Similarity >= 0.6 ---")
run_lsh_experiment_fast(0.6, exact_pairs_06)

print("\n--- Target Similarity >= 0.8 ---")
run_lsh_experiment_fast(0.8, exact_pairs_08)

--- QUESTION 5: LSH on MovieLens (OPTIMIZED) ---
Calculating exact Jaccard for 0.6 and 0.8 thresholds...

--- Target Similarity >= 0.6 ---
Config t=50, r=5, b=10: Avg FP = 3.2, Avg FN = 1.0
Config t=100, r=5, b=20: Avg FP = 0.0, Avg FN = 0.4
Config t=200, r=5, b=40: Avg FP = 0.0, Avg FN = 0.2
Config t=200, r=10, b=20: Avg FP = 0.0, Avg FN = 1.6

--- Target Similarity >= 0.8 ---
Config t=50, r=5, b=10: Avg FP = 0.2, Avg FN = 0.4
Config t=100, r=5, b=20: Avg FP = 0.0, Avg FN = 0.0
Config t=200, r=5, b=40: Avg FP = 0.0, Avg FN = 0.0
Config t=200, r=10, b=20: Avg FP = 0.0, Avg FN = 0.0
